In [1]:
import glob
import os

import cartopy
import glm as myglm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray
from glmtools.io.glm import GLMDataset
from grid.make_GLM_grids import create_parser, grid_setup

In [2]:
startdate = pd.to_datetime("20210710T07")

duration = pd.to_timedelta("1H")
enddate = startdate + duration

level2 = myglm.download(startdate, enddate, bucket="noaa-goes16")
# Load the data
glm = GLMDataset(level2[0])
print(level2)

['/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2021/191/07/OR_GLM-L2-LCFA_G16_s20211910700000_e20211910700200_c20211910700223.nc', '/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2021/191/07/OR_GLM-L2-LCFA_G16_s20211910700200_e20211910700400_c20211910700427.nc', '/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2021/191/07/OR_GLM-L2-LCFA_G16_s20211910700400_e20211910701000_c20211910701027.nc', '/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2021/191/07/OR_GLM-L2-LCFA_G16_s20211910701000_e20211910701200_c20211910701227.nc', '/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2021/191/07/OR_GLM-L2-LCFA_G16_s20211910701200_e20211910701400_c20211910701422.nc', '/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2021/191/07/OR_GLM-L2-LCFA_G16_s20211910701400_e20211910702000_c20211910702025.nc', '/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2021/191/07/OR_GLM-L2-LCFA_G16_s20211910702000_e20211910702200_c20211910702226.nc', '/glade/dere

In [3]:
g211 = cartopy.crs.LambertConformal(central_longitude=-95, standard_parallels=(25, 25))
width = 93
height = 65
llcrnrlon = -133.459
llcrnrlat = 12.19
urcrnrlon = -49.38641
urcrnrlat = 57.2894
lons = np.array([llcrnrlon, urcrnrlon])
lats = np.array([llcrnrlat, urcrnrlat])

projected_corners = g211.transform_points(cartopy.crs.PlateCarree(), lons, lats)

xs = np.linspace(projected_corners[0, 0], projected_corners[1, 0], width)
ys = np.linspace(projected_corners[0, 1], projected_corners[1, 1], height)
xv, yv = np.meshgrid(xs, ys)
llz = cartopy.crs.PlateCarree().transform_points(g211, xv, yv)
g211lon = llz[:, :, 0]
g211lat = llz[:, :, 1]

# Find flashes in some location

There are hundreds of flashes to browse above, and they are randomly scattered across the full disk. Storms near Lubbock, TX at the time of sample data file had relatively low flash rates, so let's find those.

In [4]:
lon_range = (-102, -87)
lat_range = (37, 43)
flashes_subset = glm.subset_flashes(lon_range=lon_range, lat_range=lat_range)

In [5]:
tmpdir = os.getenv("TMPDIR")
dx = 81.2705
dx = 5
(width,) = np.diff(lon_range) * 111 * np.cos(np.radians(np.array(lat_range).mean()))
(height,) = np.diff(lat_range) * 111
print(width, height)
print(width / dx, height / dx)

1275.4639977930983 666
255.09279955861967 133.2


In [7]:
parser = create_parser()
args = parser.parse_args([])
vars(args).update(
    dict(
        lcc=False,
        ctr_lat=np.array(lat_range).mean(),
        ctr_lon=np.array(lon_range).mean(),
        width=width,
        height=height,
        dx=dx,
        dy=dx,
        dt=duration.total_seconds(),
        filenames=level2,
        goes_position="east",
        min_events=1,
        min_groups=1,
        fixed_grid=False,
        subdivide_grid=1,
        goes_sector="none",
        split_events=False,
        ellipse_rev=-1,
        output_scale_and_offset=True,
        is_lma=False,
        ngroups=1,
        outdir=tmpdir,
        start=startdate,
        end=enddate,
    )
)
gridder, glm_filenames, start_time, end_time, grid_kwargs = grid_setup(args)
_ = gridder(glm_filenames, start_time, end_time, **grid_kwargs)

/glade/work/ahijevyc/conda-envs/glmval/lib/python3.12/site-packages/lmatools/coordinateSystems.py:80: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  projectedData = array(proj4.transform(self.ERSlla, self.ERSxyz, lon, lat, alt ))
/glade/work/ahijevyc/conda-envs/glmval/lib/python3.12/site-packages/lmatools/coordinateSystems.py:133: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  projectedData = array(proj4.transform(self.ERSxyz, self.projection, x, y, z ))
/glade/work/ahijevyc/conda-envs/glmval/lib/python3.12/site-packages/lmatools/coordinateSystems.py:125: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  projectedData = array(proj4.transform(self.projection, self.ERSxyz, x, y, z ))
/glade/work/ahijevyc/conda-envs/g

In [8]:
ls -l /glade/derecho/scratch/ahijevyc/tmp/GLM_20210710_070000_3600_1src_0.0586deg-dx_flash_extent.nc

-rw-r--r-- 1 ahijevyc ncar 26839 Oct 16 10:35 /glade/derecho/scratch/ahijevyc/tmp/GLM_20210710_070000_3600_1src_0.0586deg-dx_flash_extent.nc


In [27]:
# ## Grab and load the files for each grid type

nc_files = glob.glob(os.path.join(tmpdir, "GLM_*.nc"))
print(nc_files)

ged = xarray.open_mfdataset(nc_files)
ged = ged.rename(dict(ntimes="time"))
ged = ged.assign_coords(dict(lat=ged.latitude, lon=ged.longitude))
ged = ged.isel(time=0)["group_extent_density"]
ged = ged.where(ged > 0)
print(ged)

# ## MRMS

import gzip
import shutil

idir = "/glade/campaign/mmm/parc/sobash/MRMS"
glmt = glm.dataset.product_time.dt.strftime("%Y%m%d").data
search_str = f"{idir}/{glmt}"
search_str += "/MRMS_MergedReflectivityQCComposite_00.50_"
search_str += f"{glmt}*.grib2.gz"
ifiles = sorted(glob.glob(search_str))
yyyymmdd = [os.path.basename(x).replace(".grib2.gz", "")[-15:] for x in ifiles]
dbzt = pd.to_datetime(yyyymmdd)
dt = np.abs(dbzt - glm.dataset.product_time.data)
gzfile = ifiles[dt.argmin()]
print(gzfile)

base, ext = os.path.splitext(gzfile)
grb = tmpdir + "/" + os.path.basename(base)  # without .gz
if not os.path.exists(grb):
    print("gunzip")
    with gzip.open(gzfile, "rb") as f_in:
        with open(grb, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

print("opening dataset")
ds = xarray.open_dataset(grb, engine="cfgrib")
print("flipping lat")
ds = ds.reindex(latitude=ds.latitude[::-1])
print("reindexing lon")
ds = ds.assign_coords(longitude=((ds.longitude + 180) % 360) - 180)

dbz = ds.unknown.load()
dbz = dbz.where(dbz > -30)

['/glade/derecho/scratch/ahijevyc/tmp/GLM_20210710_070000_3600_1src_0.0586deg-dx_footprint.nc', '/glade/derecho/scratch/ahijevyc/tmp/GLM_20210710_070000_3600_1src_0.0586deg-dx_flash_extent.nc', '/glade/derecho/scratch/ahijevyc/tmp/GLM_20210710_070000_3600_1src_0.0586deg-dx_total_energy.nc', '/glade/derecho/scratch/ahijevyc/tmp/GLM_20210710_070000_3600_1src_0.0586deg-dx_group_area.nc', '/glade/derecho/scratch/ahijevyc/tmp/GLM_20210710_070000_3600_1src_0.0586deg-dx_flash_centroid.nc', '/glade/derecho/scratch/ahijevyc/tmp/GLM_20210710_070000_3600_1src_0.0586deg-dx_group_extent.nc', '/glade/derecho/scratch/ahijevyc/tmp/GLM_20210710_070000_3600_1src_0.0586deg-dx_group_centroid.nc']
<xarray.DataArray 'group_extent_density' (lon: 256, lat: 134)>
dask.array<where, shape=(256, 134), dtype=float32, chunksize=(256, 134), chunktype=numpy.ndarray>
Coordinates:
    time     datetime64[ns] dask.array<chunksize=(), meta=np.ndarray>
  * lat      (lat) float32 37.02 37.07 37.11 37.16 ... 42.88 42.92 42.

In [29]:
ged.lat

<xarray.DataArray 'lat' (lat: 134)>
array([37.023464, 37.068493, 37.113525, 37.158554, 37.203587, 37.248615,
       37.293648, 37.33868 , 37.38371 , 37.42874 , 37.47377 , 37.518803,
       37.56383 , 37.608864, 37.653893, 37.698925, 37.743958, 37.788986,
       37.83402 , 37.879047, 37.92408 , 37.96911 , 38.01414 , 38.05917 ,
       38.104202, 38.149235, 38.194263, 38.239296, 38.284325, 38.329357,
       38.374386, 38.41942 , 38.46445 , 38.50948 , 38.554512, 38.59954 ,
       38.644573, 38.6896  , 38.734634, 38.779663, 38.824696, 38.869728,
       38.914757, 38.95979 , 39.004818, 39.04985 , 39.09488 , 39.13991 ,
       39.18494 , 39.229973, 39.275005, 39.320034, 39.365067, 39.410095,
       39.455128, 39.500156, 39.54519 , 39.59022 , 39.63525 , 39.680283,
       39.72531 , 39.770344, 39.815372, 39.860405, 39.905434, 39.950466,
       39.9955  , 40.040527, 40.08556 , 40.13059 , 40.17562 , 40.22065 ,
       40.265682, 40.31071 , 40.355743, 40.400776, 40.445805, 40.490837,
       40.535866, 40.5809  , 40.625927, 40.67096 , 40.71599 , 40.76102 ,
       40.806053, 40.85108 , 40.896114, 40.941143, 40.986176, 41.031204,
       41.076237, 41.12127 , 41.166298, 41.21133 , 41.25636 , 41.30139 ,
       41.34642 , 41.391453, 41.43648 , 41.481514, 41.526546, 41.571575,
       41.616608, 41.661636, 41.70667 , 41.751698, 41.79673 , 41.84176 ,
       41.88679 , 41.931824, 41.976852, 42.021885, 42.066914, 42.111946,
       42.156975, 42.202007, 42.24704 , 42.29207 , 42.3371  , 42.38213 ,
       42.427162, 42.47219 , 42.517223, 42.562252, 42.607285, 42.652317,
       42.697346, 42.74238 , 42.787407, 42.83244 , 42.87747 , 42.9225  ,
       42.96753 , 43.01256 ], dtype=float32)
Coordinates:
    time     datetime64[ns] dask.array<chunksize=(), meta=np.ndarray>
  * lat      (lat) float32 37.02 37.07 37.11 37.16 ... 42.88 42.92 42.97 43.01
Attributes:
    units:          degrees_north
    long_name:      latitude
    standard_name:  latitude

In [30]:
g211lat

array([[12.19      , 12.38935861, 12.58527348, ..., 14.64692041,
        14.4938762 , 14.33711785],
       [12.8776585 , 13.07854078, 13.27594917, ..., 15.3530759 ,
        15.19889776, 15.04097539],
       [13.56665828, 13.76905356, 13.96794481, ..., 16.06042988,
        15.90512741, 15.74605077],
       ...,
       [53.40730537, 53.66363549, 53.91504821, ..., 56.53103971,
        56.33871942, 56.14141828],
       [53.98279394, 54.23926487, 54.4908056 , ..., 57.10752001,
        56.91518523, 56.71786274],
       [54.55363789, 54.81022533, 55.06187016, ..., 57.67905506,
        57.4867246 , 57.2894    ]])

In [31]:
from glmtools.plot.locations import plot_flash

In [59]:
import ipywidgets as widgets

fl_id_vals = list(glm.dataset.flash_id.data)
fl_id_vals.sort()
flash_slider = widgets.SelectionSlider(
    description="Flash",
    options=fl_id_vals,
)


def do_plot(flash_id):
    fig = plot_flash(glm, flash_id)


widgets.interact(do_plot, flash_id=flash_slider)

interactive(children=(SelectionSlider(description='Flash', options=(65032, 65033, 65034, 65036, 65037, 65038, …

<function __main__.do_plot(flash_id)>

In [33]:
from importlib import reload

import glmtools.plot.locations

reload(glmtools.plot.locations)
from glmtools.plot.locations import plot_flash

In [60]:
from glmtools.plot.values import display_params
def plot_flash(glm, flash_id, ax=None, proj=None):
    flash_data = glm.get_flashes([flash_id])

    ev_parent = flash_data.event_parent_group_id  
    gr_parent = flash_data.group_parent_flash_id


    gr_id = flash_data.group_id.data
    fl_id = flash_data.flash_id.data

    ev_lats = flash_data.event_lat.data
    ev_lons = flash_data.event_lon.data
#     ev_lats, ev_lons = fix_event_locations(flash_data.event_lat, flash_data.event_lon, is_xarray=True)
    ev_time = flash_data.event_time_offset.data
    ev_rad = flash_data.event_energy.data
    gr_lat = flash_data.group_lat.data
    gr_lon = flash_data.group_lon.data
    gr_rad = flash_data.group_energy.data
    fl_lat = flash_data.flash_lat.data
    fl_lon = flash_data.flash_lon.data
    fl_rad = flash_data.flash_energy.data
    fl_time = (flash_data.flash_time_offset_of_first_event.data[0], 
               flash_data.flash_time_offset_of_last_event.data[0])

    if ax is None:
        fig = plt.figure()
        ax_ev = fig.add_subplot(111)
    else:
        ax_ev = ax
    
    energy_norm = display_params['total_energy']['glm_norm']
    
    gr_kwargs = dict(c=gr_rad, marker='o', s=100, 
                     edgecolor='black', cmap='gray',
                     norm = energy_norm)
    if proj: gr_kwargs['transform'] = proj
    ax_ev.scatter(gr_lon, gr_lat, **gr_kwargs) 

    ev_kwargs = dict(c=ev_rad, marker='s', s=16, 
                     edgecolor='black', cmap='gray',
                     norm=energy_norm)
    if proj: ev_kwargs['transform'] = proj
    ax_ev.scatter(ev_lons, ev_lats, **ev_kwargs)
    
    fl_kwargs = dict(c='r', marker='^', s=100)
    if proj: fl_kwargs['transform'] = proj
    ax_ev.scatter(fl_lon, fl_lat, **fl_kwargs)

    ax_ev.set_title('GLM Flash #{0}\nfrom {1}\nto {2}'.format(
        fl_id[0], fl_time[0], fl_time[1]))

    return ax_ev


In [66]:
fl_id_vals = list(flashes_subset.flash_id.data)
fl_id_vals.sort()

flash_slider = widgets.IntSlider(
    description=f"{len(fl_id_vals)} flashes", min=1, max=len(fl_id_vals)
)

xlim, ylim = lon_range, lat_range
slice_dbz = dbz.sel(longitude=slice(*xlim), latitude=slice(*ylim))

state_boundaries = cartopy.feature.NaturalEarthFeature(
    category="cultural", name="admin_1_states_provinces_lakes", scale="50m", facecolor="none"
)
proj = cartopy.crs.LambertConformal(
    central_longitude=-101, central_latitude=38.33643, standard_parallels=(32, 46)
)


def do_plot(flash_id):
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    ax.add_feature(state_boundaries, linewidth=0.1, edgecolor="red")
    ax.set_extent([-103, -87, 36, 43], cartopy.crs.PlateCarree())
    slice_dbz.thin(5).plot.imshow(
        ax=ax,
        extent=(*xlim, *ylim),
        origin="lower",
        vmin=-5,
        vmax=75,
        cmap="gist_ncar",
        transform=cartopy.crs.PlateCarree(),
        cbar_kwargs=dict(shrink=0.5),
    )
    ged.plot.pcolormesh(x="lon", y="lat", ax=ax, cmap="tab20c", cbar_kwargs=dict(shrink=0.5))
    plot_flash(glm, fl_id_vals[flash_id - 1], ax=ax, proj=cartopy.crs.PlateCarree())


widgets.interact(do_plot, flash_id=flash_slider)

interactive(children=(IntSlider(value=1, description='112 flashes', max=112, min=1), Output()), _dom_classes=(…

<function __main__.do_plot(flash_id)>

In [67]:
level2 = myglm.download(
    startdate.strftime("%Y%m%d%H%M"), enddate.strftime("%Y%m%d%H%M"), bucket="noaa-goes17"
)
glm17 = GLMDataset(level2[0])
fl_id_vals17 = list(
    glm17.subset_flashes(lon_range=lon_range, lat_range=lat_range).flash_id.data
)
fl_id_vals17.sort()
flash_slider17 = widgets.IntSlider(
    description=f"{len(fl_id_vals17)} flashes", min=1, max=len(fl_id_vals17)
)


def do_plot17(flash_id):
    fig = plot_flash(glm17, fl_id_vals17[flash_id - 1])
    fig.axes.grid(alpha=0.5)
    fig.axes.set_xlim(xlim)
    fig.axes.set_ylim(ylim)
    fig.axes.imshow(slice_dbz, extent=(*xlim, *ylim), origin="lower", vmin=0, vmax=50)


_ = widgets.interact(do_plot17, flash_id=flash_slider17)

interactive(children=(IntSlider(value=1, description='58 flashes', max=58, min=1), Output()), _dom_classes=('w…